In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("puneet6060/intel-image-classification")

print("Path to dataset files:", path)

100%|██████████| 346M/346M [00:10<00:00, 35.6MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/puneet6060/intel-image-classification/versions/2


In [11]:
import tensorflow as tf
train_dir = "/root/.cache/kagglehub/datasets/puneet6060/intel-image-classification/versions/2/seg_train/seg_train"
test_dir = "/root/.cache/kagglehub/datasets/puneet6060/intel-image-classification/versions/2/seg_test/seg_test"

In [24]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(250, 250),
    batch_size=32,
    label_mode='categorical',
    color_mode='rgb'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(250, 250),
    batch_size=32,
    label_mode='categorical',
    color_mode='rgb'
)

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [14]:
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

In [25]:
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [28]:
model = models.Sequential([
    layers.Conv2D(10, (101, 101) ,activation='relu'),  # (150x150x10)
    layers.Conv2D(35, (101, 101), activation='relu'),  # (50x50x35)
    layers.MaxPooling2D((5, 5)),  # (10x10x35)
    layers.Conv2D(100, (6, 6), activation='relu'),  # (5x5x100)
    layers.MaxPooling2D((2, 2)),  # (2x2x100)
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
model.fit(train_ds, epochs=2, validation_data=test_ds)

Epoch 1/2
439/439 ━━━━━━━━━━━━━━━━━━━━ 396s 902ms/step - accuracy: 0.1879 - loss: 1.7901 - val_accuracy: 0.1670 - val_loss: 1.7906
Epoch 2/2
439/439 ━━━━━━━━━━━━━━━━━━━━ 324s 739ms/step - accuracy: 0.1821 - loss: 1.7908 - val_accuracy: 0.1750 - val_loss: 1.7904


In [31]:
test_loss, test_acc = model.evaluate(test_ds)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

94/94 ━━━━━━━━━━━━━━━━━━━━ 55s 578ms/step - accuracy: 0.1734 - loss: 1.7894
Test Accuracy: 17.50%


In [32]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 150, 150, 10)        │         306,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 50, 50, 35)          │       3,570,385 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 10, 10, 35)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 5, 5, 100)           │         126,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 2, 2, 100)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 400)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │          51,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,187,499 (46.49 MB)

 Trainable params: 4,062,499 (15.50 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,125,000 (30.99 MB)